In [1]:
from API_call import *
from functions import get_gamePks,read_gamePks
from datetime import datetime as dt
import os,re,csv
from os import walk
import statsapi as mlb
import pandas as pd
from database import *

In [2]:
get_gamePks([str(x) for x in range(2008,2020)])   

gamePks=read_gamePks()

In [3]:
call = API_call(gamePks['2019'][501])

In [4]:
db = MyDatabase('sqlite',dbname='mlb.db')

Engine(sqlite://///Users/schlinkertc/code/MLB/mlb.db)


In [5]:
Base.metadata.create_all(db.db_engine)
Base.metadata.bind=db.db_engine

In [6]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db.db_engine)
session = Session()

In [7]:
def insert(call,session):
    mappers = [
        Game,Play,
        Pitch,
        Action,Runner,Credit,
        Matchup,Venue,
        Team,GameTeamLink,TeamRecord,
        Player
    ]
    for mapper in mappers:
        table = mapper.__tablename__
        records = call.__dict__[table]
        if mapper == Runner:
            for record in records:
                if record['movement_start']==None:
                    record['movement_start']='Home'
        session.bulk_insert_mappings(mapper,records)
    for mapper in [PitchData,HitData]:
        cols = [x.name for x in mapper.__table__.c]
        records = call.__dict__['pitches']
        df = pd.DataFrame.from_records(records)[cols]
        df.dropna(axis=0,thresh=5,inplace=True)
        session.bulk_insert_mappings(mapper,df.to_dict(orient='records'))
    session.commit()
        
        

In [8]:
insert(call,session)

//anaconda3/lib/python3.7/site-packages/sqlalchemy/sql/crud.py:799: SAWarning: Column 'pitch_data.gamePk' is marked as a member of the primary key for table 'pitch_data', but has no Python-side or server-side default generator indicated, nor does it indicate 'autoincrement=True' or 'nullable=True', and no explicit value is passed.  Primary key columns typically may not store NULL. Note that as of SQLAlchemy 1.1, 'autoincrement=True' must be indicated explicitly for composite (e.g. multicolumn) primary keys if AUTO_INCREMENT/SERIAL/IDENTITY behavior is expected for one of the columns in the primary key. CREATE TABLE statements are impacted by this change as well on most backends.
  util.warn(msg)
//anaconda3/lib/python3.7/site-packages/sqlalchemy/sql/crud.py:799: SAWarning: Column 'pitch_data.atBatIndex' is marked as a member of the primary key for table 'pitch_data', but has no Python-side or server-side default generator indicated, nor does it indicate 'autoincrement=True' or 'nullabl

IntegrityError: (sqlite3.IntegrityError) NOT NULL constraint failed: pitch_data.gamePk
[SQL: INSERT INTO pitch_data DEFAULT VALUES]
[parameters: ((), (), (), (), (), (), (), ()  ... displaying 10 of 284 total bound parameter sets ...  (), ())]
(Background on this error at: http://sqlalche.me/e/gkpj)

In [ ]:
# session.bulk_insert_mappings(Game,call.game)
# session.bulk_insert_mappings(Play,call.plays)
# session.bulk_insert_mappings(Pitch,call.pitches)
# session.bulk_insert_mappings(HitData,call.pitches)
# session.bulk_insert_mappings(PitchData,call.pitches)

# session.commit()

In [ ]:
pitches = pd.read_sql('select * from pitches',db.db_engine)
hitData = pd.read_sql('select * from hit_data',db.db_engine)
pitchData = pd.read_sql('select * from pitch_data',db.db_engine)

In [ ]:
games = session.query(Game).all()
plays = session.query(Play).all()

In [9]:
cols = [x.name for x in PitchData.__table__.c]

In [10]:
pitchData = pd.DataFrame.from_records(call.pitches)[cols]

In [11]:
# good to know for later 
pitchData.dropna(axis=0,thresh=5,inplace=True)

In [13]:
pitchData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284 entries, 0 to 286
Data columns (total 29 columns):
gamePk                            284 non-null int64
atBatIndex                        284 non-null int64
playEndTime                       284 non-null object
index                             284 non-null int64
pitchData_startSpeed              284 non-null float64
pitchData_endSpeed                284 non-null float64
pitchData_strikeZoneTop           284 non-null float64
pitchData_zone                    284 non-null float64
pitchData_typeConfidence          284 non-null float64
pitchData_plateTime               284 non-null float64
pitchData_extension               284 non-null float64
pitchData_coordinates_aY          284 non-null float64
pitchData_coordinates_aZ          284 non-null float64
pitchData_coordinates_pfxX        284 non-null float64
pitchData_coordinates_pfxZ        284 non-null float64
pitchData_coordinates_pX          284 non-null float64
pitchData_coordinates_

In [14]:
test_games = gamePks['2019'][600:650]

calls = [ API_call(g) for g in test_games ]

In [26]:
dfs = {}
for k in calls[0].__dict__.keys():
    if k[0] != '_':
        records = [x for call in calls for x in call.__dict__[k]]
        df = pd.DataFrame.from_records(records)
        dfs[k] = df

In [27]:
pitches = dfs['pitches']

In [28]:
pitches = pitches[cols].dropna(axis=0,thresh=5)

In [37]:
pitches.index.name = 'INDEX'

In [38]:
pitches.to_records()

rec.array([(    0, 567378,  0, '2019-05-04T22:11:11.000Z', 3, 91.3, 83. , 3.2 , 11.,  2., 0.41, 6.18, 29.71, -12.41, -6.2 , 11.11, -1.21, 4.  ,  2.58, -132.88, -3.24, 163.19, 130.76, -1.39, 50., 6.14, -11.03, 24., 2168., 208.),
           (    1, 567378,  0, '2019-05-04T22:11:11.000Z', 4, 91.5, 83.2, 3.29, 11.,  2., 0.41, 6.4 , 30.12, -12.61, -4.65, 10.96, -0.15, 3.3 ,  4.92, -133.14, -4.77, 122.51, 149.59, -1.42, 50., 6.04,  -8.3 , 24., 2205., 200.),
           (    2, 567378,  0, '2019-05-04T22:11:11.000Z', 5, 80.4, 74.1, 3.22, 13.,  2., 0.47, 5.93, 23.34, -36.31,  0.31, -3.01, -0.19, 0.48,  3.01, -116.86, -4.63, 124.25, 225.9 , -1.54, 50., 5.92,   0.43, 24., 2331.,  12.),
           ...,
           (15472, 565429, 84, '2019-05-08T04:05:21.000Z', 1, 93.7, 87.2, 3.36,  5., nan, 0.4 , 5.64, 24.71, -20.17,  3.12,  6.26,  0.07, 2.71, -6.62, -136.33, -4.5 , 114.46, 165.71,  2.1 , 50., 5.74,   5.99, 24., 2152., 157.),
           (15473, 565429, 85, '2019-05-08T04:06:18.000Z', 0, 87.3, 81.6

In [ ]:
dfs.keys()

In [ ]:
runners = dfs['runners']

In [ ]:
runners.info()

In [ ]:
pks = [
    'atBatIndex','playEndTime','gamePk',
    'details_playIndex','details_runner_id','details_eventType','movement_start'
]

In [ ]:
runners['movement_start'].fillna('home').value_counts().sum()

In [ ]:
runners[pks][runners[pks].duplicated()==True]

In [ ]:
runners[(runners['gamePk']==567281)&(runners['atBatIndex']==57)]

In [ ]:
runners[['atBatIndex','playEndTime','gamePk','details_playIndex','details_runner_id','movement_end']].duplicated()

In [ ]:
tr = dfs['team_records']

In [ ]:
tr.info()

In [ ]:
tr.head()